In [1]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import UnstructuredFileLoader
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100
)

loader = UnstructuredFileLoader("document.txt")
docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()
cache = LocalFileStore("./.cache/")
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings,
    cache
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)
retriever = vectorstore.as_retriever()

In [13]:
from operator import itemgetter
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="history", return_messages=True)
runnable_history = RunnablePassthrough.assign(
    history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
)

prompt = ChatPromptTemplate.from_messages([
    (
        "system", 
        """
        You are a helpful assistant that can answer questions using only the provided contexts and conversation history.
        If you don't know the answer, just say you don't know. Don't make up an answer.
        -----
        {context}
        """
    ),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}")
])

llm = ChatOpenAI(temperature=0)

chain = (
    {"context": retriever, "question": RunnablePassthrough()} | 
    runnable_history | 
    prompt | 
    llm
)

In [14]:
def invoke_chain(question):
    print("Question: ", question)
    result = chain.invoke(question)
    print("Answer: ", result.content)
    memory.save_context({"input": question}, {"output": result.content})

invoke_chain("Is Aaronson guilty?")
invoke_chain("What message did he write in the table?")
invoke_chain("Who is Julia?")

Question:  Is Aaronson guilty?
Answer:  According to the document, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.
Question:  What message did he write in the table?
Answer:  The message he wrote on the table was "2+2=5."
Question:  Who is Julia?
Answer:  Julia is a character in the provided text who is associated with the protagonist and plays a significant role in the story.
